# Import processed data from csv and building clustering model

## Imports and global declarations

In [1]:
!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error

from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import glob
import re
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 


## Reading processed data from a csv file

In [25]:
# . is any character except new line, he.{2} all should match where we have 2 characters after he
# .*he mean any number of character before he
a = [re.search(r'\d{2}-\d{2}-\d{2}', x).group(0) for x in glob.glob("../data/processed/1.2-um-data-process-all*")]
b = sorted([dt.datetime.strptime(x,"%d-%m-%y") for x in a])
file_path=[val for val in glob.glob("../data/processed/1.2-um-data-process-all*") if re.match('.*' + b[-1].strftime('%d-%m-%y'),val)][0]
print(f"Loading csv file from {file_path}")
global_data = pd.read_csv(file_path, index_col=0)

Loading csv file from ../data/processed/1.2-um-data-process-all-01-08-22.csv


##  Building knn model and storing model, label and clustered data in local files

In [26]:
X = global_data.loc[:, ~global_data.columns.isin(['dev_id', 'paying_cust'])]
Y = global_data['paying_cust']
dev_id = global_data['dev_id']                                      
kmeanModel = KMeans(n_clusters=5, init='k-means++', verbose=0, random_state=13, n_init=20)
kmeanModel = kmeanModel.fit(X)
global_data['cluster'] = kmeanModel.predict(X)
#X['cluster2'] = kmeanModel.labels_
#sum(X['label'] == X['cluster'])

now = dt.datetime.now()
current_time = now.strftime("%d-%m-%y") # %H:%M:%S")
filename = '../models/2.0-knn-clustering-model-' + current_time+ '.sav'
pickle.dump(kmeanModel, open(filename, 'wb'))


cluster_info = global_data.groupby('cluster').agg({'paying_cust':'mean'}).reset_index().sort_values(by='paying_cust', ascending=True).reset_index()
cluster_label = {cluster_info.at[4, 'cluster']:'Elite',
                 cluster_info.at[3, 'cluster']:'High Quality',
                 cluster_info.at[2, 'cluster']:'Average',
                 cluster_info.at[1, 'cluster']:'Low Quality',
                 cluster_info.at[0, 'cluster']:'Low Chance'}

filename = '../models/2.0-knn-cluster-label-' + current_time+ '.pckl' 
pickle.dump(cluster_label, open(filename, 'wb'))

global_data['label'] = global_data['cluster'].map(cluster_label)
global_data.to_csv('../data/processed/' + 'ml_clustered_data ' + current_time+'.csv')
